Import dependencies

In [113]:
import random
from shapely.geometry import Polygon, Point
from k_means_constrained import KMeansConstrained
from ipyleaflet import (
    Map,
    DrawControl,
    Marker,
    AwesomeIcon,
    Icon
)
from time import time
from math import floor

Set Şerifali polygon for random geolocation generation.

In [ ]:
poly = Polygon([[29.1383021, 40.996777], [29.1392317, 40.9966518], [29.1396906, 40.9965677], [29.1464479, 40.9953285],
                [29.1484288, 40.9950402], [29.1508561, 40.9947584], [29.1524178, 40.9946703], [29.1529304, 40.994669],
                [29.1549765, 40.994664], [29.1553184, 40.9946632], [29.1552472, 40.9954192], [29.155156, 40.9962492],
                [29.1549253, 40.9974476], [29.1548175, 40.9977817], [29.1548181, 40.9979932], [29.1548618, 40.9981628],
                [29.1550613, 40.9986191], [29.1554278, 40.9992955], [29.1555865, 40.9996108], [29.1560911, 41.0006134],
                [29.156274, 41.0008778], [29.1566205, 41.0013788], [29.1567892, 41.0015746], [29.1565052, 41.0017209],
                [29.1562504, 41.0018404], [29.1557649, 41.0020367], [29.1553035, 41.0022067], [29.1540724, 41.0027128],
                [29.1523853, 41.0033707], [29.1522029, 41.0034435], [29.1505855, 41.0040832], [29.1489682, 41.0047349],
                [29.148362, 41.0049637], [29.1468787, 41.0055567], [29.1465153, 41.0057065], [29.1461827, 41.0058462],
                [29.1460673, 41.0059282], [29.1459882, 41.0060223], [29.145952, 41.0059828], [29.1456771, 41.0058098],
                [29.1455269, 41.0057025], [29.1453284, 41.0055648], [29.1450199, 41.0053483], [29.1448134, 41.0051883],
                [29.1446954, 41.0050811], [29.144403, 41.0048017], [29.1441509, 41.0045426], [29.1438693, 41.0042491],
                [29.1430056, 41.0033524], [29.1422626, 41.0026197], [29.1418657, 41.0022128], [29.1418132, 41.0021467],
                [29.1417785, 41.002103], [29.1417309, 41.0020272], [29.1416994, 41.001978], [29.1415706, 41.0017341],
                [29.1413386, 41.0013798], [29.1411951, 41.0010995], [29.1403589, 41.0010912], [29.1403099, 41.0010914],
                [29.1401866, 41.0006359], [29.1401222, 41.0003667], [29.1400927, 41.0001643], [29.1401222, 40.9999355],
                [29.1402027, 40.9990995], [29.1400632, 40.9986177], [29.1399211, 40.99823], [29.1397935, 40.9978893],
                [29.1397534, 40.9977867], [29.139622, 40.9974861], [29.1395115, 40.99725], [29.1394865, 40.9971966],
                [29.1383021, 40.996777]])
# Make poly shrink to not intercept with highway
poly = poly.buffer(-0.0005, join_style=2)
print(poly)

Generate Data

In [ ]:
min_x, min_y, max_x, max_y = poly.bounds
points = []
while len(points) < 500:
    random_point = Point([random.uniform(min_x, max_x), random.uniform(min_y, max_y)])
    if random_point.within(poly):
        points.append(random_point)
# Choose the number of points desired. This example uses 20 points. 
# Printing the results.
for p in points:
    print(p.x, ",", p.y)
# Credits https://medium.com/the-data-journal/a-quick-trick-to-create-random-lat-long-coordinates-in-python-within-a-defined-polygon-e8997f05123a

Display Points on Map

In [ ]:
m = Map(zoom=16)
markers = []
for p in points:
    icon = Icon(icon_url="./imgs/icons8-dot-24.png", icon_size=(10, 10))
    marker = Marker(location=(p.y, p.x), draggable=False, icon=icon)
    m.add(marker)
    markers.append(marker)

dc = DrawControl()

m.add_control(dc)
m.center = (points[0].y, points[0].x)
m

Define warehouse location

In [ ]:
warehouseLocation = (40.99808944128814, 29.15221946728115)

warehouseMarker = Marker(z_index_offset=100,
                         location=warehouseLocation,
                         icon=Icon(icon_url='./imgs/icons8-warehouse-48.png',
                                   marker_color='red',
                                   icon_color='black'))

m.add(warehouseMarker)
m.center = warehouseLocation
m

Cluster points.

In [114]:
# Change me
total_basket = 10

colors = ['white', 'red', 'darkred', 'lightred', 'orange', 'beige', 'green', 'darkgreen', 'lightgreen', 'blue',
          'darkblue', 'lightblue', 'purple', 'darkpurple', 'pink', 'cadetblue', 'white', 'gray', 'lightgray', 'black']

start_time = time()
kmeans = KMeansConstrained(
    n_clusters=20,
    size_min=18,
    size_max=floor(len(points) / 20),
    random_state=0
).fit(list(map(lambda point: (point.x, point.y), points)))
end_time = time() - start_time
print("Execution Time:" + str(end_time))
labels = kmeans.labels_

m = Map(zoom=16)
for n in range(len(points)):
    point = points[n]
    label = labels[n]
    marker = Marker(location=(point.y, point.x), draggable=False,
                    icon=AwesomeIcon(marker_color=colors[label],
                                     ))
    m.add(marker)

dc = DrawControl()

m.add_control(dc)
m.center = (points[0].y, points[0].x)
m

Execution Time:14.911939859390259


Map(center=[41.00038494800897, 29.150258608111837], controls=(ZoomControl(options=['position', 'zoom_in_text',…